<h1>Text Classification using Naive Bayes<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span></li><li><span><a href="#Essential-Concepts" data-toc-modified-id="Essential-Concepts-2">Essential Concepts</a></span><ul class="toc-item"><li><span><a href="#Naive-Bayes-Theory" data-toc-modified-id="Naive-Bayes-Theory-2.1">Naive Bayes Theory</a></span></li><li><span><a href="#Numpy" data-toc-modified-id="Numpy-2.2">Numpy</a></span></li><li><span><a href="#SKLearn" data-toc-modified-id="SKLearn-2.3">SKLearn</a></span></li></ul></li><li><span><a href="#Machine-Learning-Project-Lifecycle:-First-Iteration" data-toc-modified-id="Machine-Learning-Project-Lifecycle:-First-Iteration-3">Machine Learning Project Lifecycle: First Iteration</a></span><ul class="toc-item"><li><span><a href="#Problem-Statement" data-toc-modified-id="Problem-Statement-3.1">Problem Statement</a></span></li><li><span><a href="#Training-Data" data-toc-modified-id="Training-Data-3.2">Training Data</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-3.3">Preprocessing</a></span></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-3.4">Feature Engineering</a></span></li><li><span><a href="#Machine-Learning-Algorithm:-Naive-Bayes" data-toc-modified-id="Machine-Learning-Algorithm:-Naive-Bayes-3.5">Machine Learning Algorithm: Naive Bayes</a></span></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-3.6">Modeling</a></span></li><li><span><a href="#Quality-Metrics" data-toc-modified-id="Quality-Metrics-3.7">Quality Metrics</a></span></li><li><span><a href="#Model-Evaluation" data-toc-modified-id="Model-Evaluation-3.8">Model Evaluation</a></span></li></ul></li><li><span><a href="#Homework" data-toc-modified-id="Homework-4">Homework</a></span></li><li><span><a href="#Resources" data-toc-modified-id="Resources-5">Resources</a></span></li></ul></div>

## Introduction

## Essential Concepts

### Naive Bayes Theory

### Numpy

### SKLearn

## Machine Learning Project Lifecycle: First Iteration

### Problem Statement

### Training Data

### Preprocessing

### Feature Engineering

### Machine Learning Algorithm: Naive Bayes

### Modeling

### Quality Metrics

### Model Evaluation

## Homework

## Resources